<h1 style="color: skyblue"> Deeptech Engineer Intern Challenge </h1>

In [129]:
# imports
import pandas as pd
from fuzzywuzzy import process
import random

In [130]:
# Facebook dataset
chunk_size = 1000000 
facebook_dataset = pd.read_csv('facebook_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
facebook_dataset.head(5)

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,NaN,occ,occitanie,30420
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,ca,NaN,NaN,NaN
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,ca,mb,manitoba,r3l 2t7


In [131]:
# Google dataset

google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
google_dataset.head(5)

/tmp/ipykernel_14073/98803827.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')


,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,+61243355946,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,st. catharines,ca,canada,Heritage Christian Book Store,+19059374553,ca,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,yarmouth,ca,canada,Pleasant Timber Mart,+19027429181,ca,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,los angeles,us,united states,Skin Specifics Medical Spa West LA,+18184268353,us,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee
4,NaN,Catering & Delivery,NaN,NaN,NaN,Lakay express,NaN,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee


In [132]:
# Website dataset

website_dataset = pd.read_csv('website_dataset.csv', delimiter=';', quotechar='"', on_bad_lines='skip')
website_dataset.head(5)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [133]:
# common columns

columns_facebook = set(facebook_dataset.columns)
columns_google = set(google_dataset.columns)
columns_website = set(website_dataset.columns)


# function to find similar columns
def find_similar_columns(columns1, columns2, columns3):
    similar_columns = []
    for col1 in columns1:
        # find the most similar column in columns2 and columns3
        match2 = process.extractOne(col1, columns2)
        match3 = process.extractOne(col1, columns3)
        # consider a match if similarity score is above a threshold (80)
        if match2[1] > 80 and match3[1] > 80:
            similar_columns.append((col1, match2[0], match3[0]))
    return similar_columns

# find similar columns across the datasets
similar_columns = find_similar_columns(columns_facebook, columns_google, columns_website)

# print the similar columns
for col1, col2, col3 in similar_columns:
    print(f"Similar Columns: {col1}, {col2}, {col3}")

Similar Columns: phone, phone, phone
Similar Columns: phone_country_code, phone_country_code, phone
Similar Columns: name, name, legal_name
Similar Columns: city, city, main_city
Similar Columns: domain, domain, root_domain


In [134]:
# modify root_domain with doman in website dataset
website_dataset.rename(columns = {'root_domain': 'domain'}, inplace = True)
website_dataset.head(5)

,domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [135]:
# make a new DataFrame

merged_df = pd.merge(facebook_dataset['domain'], google_dataset['domain'], on = 'domain', how = 'outer')
merged_df = pd.merge(merged_df, website_dataset['domain'], on = 'domain', how = 'outer')

# drop duplicates
merged_df.drop_duplicates(inplace = True)

# make domain as primary key / id
merged_df.set_index('domain', inplace = True)
merged_df.sample(5)

""
domain
prostudiomtl.com
tbmtc.ca
westondetroit.com
nkolayislem.com.tr
scoreondavie.com


In [136]:
# add to dataframe following common columns: Category, Address, Phone, Company names.

# first preprocess the categories to have the same column name

# category column name
facebook_dataset.rename(columns = {'categories': 'category'}, inplace = True)
website_dataset.rename(columns = {'s_category': 'category'}, inplace = True)

In [137]:
facebook_dataset['category'].sample(5)

67407    Clothing Stores|Fashion Designers|Social Servi...
20476    Radio & Television Production|Cinemas & Video ...
65032    Utility companies|Utility companies|Community ...
39994                                                  NaN
19835                                                  NaN
Name: category, dtype: object

In [138]:
website_dataset['category'].sample(5)

47405    Executive, Legislative & Government Institutions
2456                                Social Media Agencies
63108                                Solar Energy Company
37068                                      Parked Domains
14636                            Watches & Jewelry Stores
Name: category, dtype: object

In [139]:
# address is missing from website dataset

# phone is the same

# website dataset is different, should be name
website_dataset.rename(columns = {'legal_name': 'name'}, inplace = True)
website_dataset['name'].sample(5)

68095    Precision Protection Systems Ltd.
31778                                  NaN
17518                 Bogdane Trucking Ltd
3587                                   NaN
37363               Cyno Luc Campbell inc.
Name: name, dtype: object

In [140]:
# drop duplicates from all datasets
website_dataset.drop_duplicates(inplace = True)
facebook_dataset.drop_duplicates(inplace = True)
google_dataset.drop_duplicates(inplace = True)

In [141]:
# merge category column
merged_category = pd.merge(facebook_dataset[['domain', 'category']], google_dataset[['domain', 'category']], on = "domain")
merged_category = pd.merge(merged_category, website_dataset[['domain', 'category']], on = "domain")
merged_category.rename(columns = {'category': 'category_z'}, inplace = True)
merged_category.drop_duplicates(inplace  = True)
merged_category.sample(5)

,domain,category_x,category_y,category_z
117291,kingstonoliveoil.com,Catering & Delivery|Fruit & Vegetable - Market...,Fruit & Vegetable - Markets & Stores,Dairy Products - Farms & Stores
2167,ccdenver.org,Social Services & NGOs|Social Services & NGOs,Social Services & NGOs,Social Services & NGOs
22900,bit.ly,Sporting Goods Store|Sporting Goods Store,Events Venues,Tech Companies
28805,hetnieuweinstituut.nl,Museums,Museums,Museums
289217,vickarchevrolet.ca,Automobile Dealers & Manufacturers|Automobile ...,Automobile Dealers & Manufacturers,Automobile Dealers & Manufacturers


In [142]:
# merge address column
merged_address = pd.merge(facebook_dataset[['domain', 'address']], google_dataset[['domain', 'address']], on = "domain")
merged_address.rename(columns = {'address': 'address_z'}, inplace = True)
merged_address.drop_duplicates(inplace  = True)
merged_address.sample(5)

,domain,address_x,address_y
284701,hilton.com,"7930 jones branch dr, 22102, mclean, va, virginia",Period house with a striking conservatory and ...
306943,economik.com,"215 st-augustin, h4c 2n7, montréal, québec, ca...",215 St Augustin St
112889,ucihealth.org,"101 the city drive south, 92868, orange, ca, c...","1002 Health Sciences Rd, Irvine, CA 92617"
171344,wvi.org,NaN,"Addis Ababa, Ethiopia"
284986,hilton.com,"7930 jones branch dr, 22102, mclean, va, virginia","Relaxed quarters, some with river views, in a ..."


In [143]:
# concat category and address into a new dataframe
# DF = pd.merge(merged_category, merged_address, on = 'domain')
# DF.drop_duplicates(inplace = True)
# DF.head(5)
DF = pd.concat([merged_category, merged_address], axis = 0)
DF.head(5)

,domain,category_x,category_y,category_z,address_x,address_y
0,euro-hygiene-34.fr,NaN,Pest Services & Agricultural Chemicals,Pest Services & Agricultural Chemicals,NaN,NaN
1,lakesidehomeservices.ca,Appliance Repair & Maintenance|Home Builders &...,Appliance Repair & Maintenance,Appliance Repair & Maintenance,NaN,NaN
2,rossiterboats.com,Boats & Yachts Dealers|Boat Tours & Cruises,Boats & Yachts Dealers,Boats & Yachts Repair Shops,NaN,NaN
3,caa-architecture.com,Architects & Architectural Services|Other Engi...,Architects & Architectural Services,Architects & Architectural Services,NaN,NaN
4,apexsurety.ca,NaN,"Insurance - Agents, Carriers & Brokers",Liability Insurance Agency,NaN,NaN


In [144]:
# merge phone column
merged_phone = pd.merge(facebook_dataset[['domain', 'phone']], google_dataset[['domain', 'phone']], on = "domain")
merged_phone = pd.merge(merged_phone, website_dataset[['domain', 'phone']], on = "domain")
merged_phone.rename(columns = {'phone': 'phone_z'}, inplace = True)
merged_phone.drop_duplicates(inplace  = True)
merged_phone.sample(5)

,domain,phone_x,phone_y,phone_z
327513,mapremiereacademiesportive.com,NaN,14509660611.0,14509660611
124086,carquest.com,1.613925e+10,+18036252500,19734231700
234429,facebook.com,NaN,+441418874560,18772765554
225270,facebook.com,NaN,+84399555516,18772765554
186338,gfp.sd.gov,1.605773e+10,16052963643.0,16052237660


In [145]:
# concat DF and phone into a new dataframe
DF = pd.concat([DF, merged_phone], axis = 0)
DF.head(5)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z
0,euro-hygiene-34.fr,NaN,Pest Services & Agricultural Chemicals,Pest Services & Agricultural Chemicals,NaN,NaN,NaN,NaN,NaN
1,lakesidehomeservices.ca,Appliance Repair & Maintenance|Home Builders &...,Appliance Repair & Maintenance,Appliance Repair & Maintenance,NaN,NaN,NaN,NaN,NaN
2,rossiterboats.com,Boats & Yachts Dealers|Boat Tours & Cruises,Boats & Yachts Dealers,Boats & Yachts Repair Shops,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,Architects & Architectural Services|Other Engi...,Architects & Architectural Services,Architects & Architectural Services,NaN,NaN,NaN,NaN,NaN
4,apexsurety.ca,NaN,"Insurance - Agents, Carriers & Brokers",Liability Insurance Agency,NaN,NaN,NaN,NaN,NaN


In [146]:
# merge company column
merged_company = pd.merge(facebook_dataset[['domain', 'name']], google_dataset[['domain', 'name']], on = "domain")
merged_company = pd.merge(merged_company, website_dataset[['domain', 'name']], on = "domain")
merged_company.rename(columns = {'name': 'name_z'}, inplace = True)
merged_company.drop_duplicates(inplace  = True)
merged_company.sample(5)

,domain,name_x,name_y,name_z
258663,facebook.com,Facebook,Pacific Ocean Harvesters,Facebook Ireland Ltd.
138035,yorkregion.com,Atlas Landscape Inc.,York Region Media Group,OwnLocal Inc.
125380,napacanada.com,Woody's Automotive Ltd - Napa Auto Parts Stettler,NAPA Pièces d'auto - Pièces d'auto du Suroît,UAP Inc.
169895,lr-doan-flooring.business.site,L&R Doan Flooring,L&R Doan Flooring,NaN
315942,goarmy.com,US Army Recruiting Glen Burnie,Army Recruiting Spanish Fort,Sears Holdings Corporation


In [147]:
# concat DF and name into a new dataframe
DF = pd.concat([DF, merged_company], axis = 0)
DF.sample(10)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z,name_x,name_y,name_z
248240,facebook.com,NaN,NaN,NaN,NaN,"37 Sciarra Cres, Acacia Gardens NSW 2763, Aust...",NaN,NaN,NaN,NaN,NaN,NaN
1142,bu.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Questrom School of Business, Boston University",Feld Career Center,NaN
40947,housedoctors.com,NaN,NaN,NaN,NaN,NaN,1.214433e+10,16147452828.0,18003193359,NaN,NaN,NaN
297346,fcphd.org,NaN,NaN,NaN,"36 klondike rd, 99166, republic, wa, washington","36 Klondike Rd, Republic, WA 99166",NaN,NaN,NaN,NaN,NaN,NaN
292236,philhealth.gov.ph,NaN,NaN,NaN,NaN,NaN,NaN,+63495459356,6324417442,NaN,NaN,NaN
64371,schools.cms.k12.nc.us,NaN,NaN,NaN,"7600 ibm dr, 28262, charlotte, nc, north carolina","7905 Pleasant Grove Rd, Charlotte, NC 28216",NaN,NaN,NaN,NaN,NaN,NaN
169784,sandiego.gov,NaN,NaN,NaN,NaN,"7603803800, San Diego, CA 92109, United States",NaN,NaN,NaN,NaN,NaN,NaN
152043,ochsner.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ochsner Health,"Kristen F. Gradney, RD",Ochsner Company
68374,pvoil.com.vn,NaN,NaN,NaN,"tòa nhà petrovietnam, 1-5 lê duẩn, quận 1, ho ...","An Giang Province, Vietnam · +84 96 442 28 33",NaN,NaN,NaN,NaN,NaN,NaN
10190,maine.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maine Emergency Management Agency (MEMA),Hamlin Town Hall,"Tyler Technologies, Inc."


In [148]:
#  category from category_x	category_y	category_z	with those conditions: if two of them has same values, assign the category column that value,
# if there are all different, assign the category column a value from category_x	category_y	category_z	based on other same categories like
#  address_x	address_y phone_x	phone_y	phone_z	 name_x	name_y	name_z or have common words or if all are NaN assign NaN or if one is non nan assign it 
# (do this for address from 'address_x', 'address_y', phone from 'phone_x', 'phone_y', 'phone_z' and name from name_x', 'name_y',  'name_z' too)
# if there after all the conditions the result is NaN then assign it with a random no nan value from column if exists
# if there are just one non na value than assign the column to it

# function to check for common words

def common_words(row, features):
    for cat in features:
        for other_cat in features:
            if cat != other_cat:
                val1 = str(row[cat]) if not pd.isnull(row[cat]) else ''
                val2 = str(row[other_cat]) if not pd.isnull(row[other_cat]) else ''
                common = set(val1.split()).intersection(val2.split())
                if common:
                    return common.pop()  # return the common word
    return None

# function to assign category based on conditions
def assign_category(row, categories):
    values = [row[cat] for cat in categories]
    non_nan_values = [val for val in values if not pd.isnull(val)]
    
    if len(non_nan_values) == 1:
        return non_nan_values[0]
    elif len(set(values)) == 1:  # all categories have the same value
        return values[0]
    elif non_nan_values:
        common_word = common_words(row, categories)
        if common_word:
            for cat in categories:
                if common_word in str(row[cat]).split():
                    return row[cat]
        else:
            for cat in categories:
                if values.count(row[cat]) > 1:  # duplicate values in other categories
                    return row[cat]
    else:
        non_nan_values = [val for val in row[categories] if not pd.isnull(val)]
        if non_nan_values:
            return random.choice(non_nan_values)
        else:
            return None

# function to assign address based on conditions
def assign_address(row, addresses):
    values = [row[cat] for cat in addresses]
    non_nan_values = [val for val in values if not pd.isnull(val)]
    
    if len(non_nan_values) == 1:
        return non_nan_values[0]
    elif len(set(values)) == 1:  # all categories have the same value
        return values[0]
    elif non_nan_values:
        common_word = common_words(row, addresses)
        if common_word:
            for cat in addresses:
                if common_word in str(row[cat]).split():
                    return row[cat]
        else:
            for cat in addresses:
                if values.count(row[cat]) > 1:  # duplicate values in other categories
                    return row[cat]
            if len(non_nan_values) == 2:  # only two different non-NaN values
                return non_nan_values[0]  # pick one to minimize NaN
    else:
        non_nan_values = [val for val in row[addresses] if not pd.isnull(val)]
        if non_nan_values:
            return random.choice(non_nan_values)
        else:
            return None

# Function to assign phone based on conditions
def assign_phone(row, phones):
    values = [row[cat] for cat in phones]
    non_nan_values = [val for val in values if not pd.isnull(val)]
    
    if len(non_nan_values) == 1:
        return non_nan_values[0]
    elif len(set(values)) == 1:  # All categories have the same value
        return values[0]
    elif non_nan_values:
        common_word = common_words(row, phones)
        if common_word:
            for cat in phones:
                if common_word in str(row[cat]).split():
                    return row[cat]
        else:
            for cat in phones:
                if values.count(row[cat]) > 1:  # If there are duplicate values in other categories
                    return row[cat]
            if len(non_nan_values) == 2:  # If there are only two different non-NaN values
                return non_nan_values[0]  # Pick one to minimize NaNs
    else:
        non_nan_values = [val for val in row[phones] if not pd.isnull(val)]
        if non_nan_values:
            return random.choice(non_nan_values)
        else:
            return None

# function to assign name based on conditions
        
def assign_name(row, names):
    values = [row[cat] for cat in names]
    non_nan_values = [val for val in values if not pd.isnull(val)]
    
    if len(non_nan_values) == 1:
        return non_nan_values[0]
    elif len(set(values)) == 1:  # all categories have the same value
        return values[0]
    elif non_nan_values:
        common_word = common_words(row, names)
        if common_word:
            for cat in names:
                if common_word in str(row[cat]).split():
                    return row[cat]
        else:
            for cat in names:
                if values.count(row[cat]) > 1:  # duplicate values in other categories
                    return row[cat]
            if len(non_nan_values) == 2:  # only two different non-NaN values
                return non_nan_values[0]  # pick one to minimize NaNs
    else:
        non_nan_values = [val for val in row[names] if not pd.isnull(val)]
        if non_nan_values:
            return random.choice(non_nan_values)
        else:
            return None

In [149]:
category_cols = ['category_x', 'category_y', 'category_z']
address_cols = ['address_x', 'address_y']
phone_cols = ['phone_x', 'phone_y', 'phone_z']
name_cols = ['name_x', 'name_y', 'name_z']

DF['category'] = DF.apply(lambda row: assign_category(row, category_cols), axis=1)
DF['address'] = DF.apply(lambda row: assign_address(row, address_cols), axis=1)
DF['phone'] = DF.apply(lambda row: assign_phone(row, phone_cols), axis=1)
DF['name'] = DF.apply(lambda row: assign_name(row, name_cols), axis=1)

DF.sample(5)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z,name_x,name_y,name_z,category,address,phone,name
223372,facebook.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Facebook,Angie's Pet Spa,Facebook Ireland Ltd.,NaN,NaN,None,Facebook
44130,hiwaygreenhouse.ca,Garden Equipment & Supplies,Plant Nurseries & Stores,Plant Nurseries & Stores,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden Equipment & Supplies,NaN,None,NaN
69364,jenganllc.com,NaN,NaN,NaN,"al nahda 1 dubai, 255, dubai, united arab emir...",JENGAN LLC (Al Ain Branch) - 24°11'47.0\N 55°4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"al nahda 1 dubai, 255, dubai, united arab emir...",None,NaN
125165,napacanada.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Woody's Automotive Ltd - Napa Auto Parts Stettler,McIlwaine Services,UAP Inc.,NaN,NaN,None,None
186644,platoscloset.com,NaN,NaN,NaN,NaN,"2480 Briarcliff Rd NE #2, Atlanta, GA 30329",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2480 Briarcliff Rd NE #2, Atlanta, GA 30329",None,NaN


In [150]:
DF.drop(columns = ['category_x', 'category_y', 'category_z', 'address_x', 'address_y', 'phone_x', 'phone_y', 'phone_z', 'name_x', 'name_y', 'name_z'], inplace = True)

In [176]:
# final DF
DF.rename(columns = {'name': 'company name'}, inplace = True)
DF = DF[['domain', 'company name', 'category', 'address', 'phone']]
DF.sample(10)

,domain,company name,category,address,phone
79330,dnhc.ca,NaN,Alternative Therapy|Clinics - Surgeons & Physi...,NaN,None
203462,facebook.com,NaN,NaN,NaN,+18326888743
134718,cmlibrary.org,Charlotte Mecklenburg Library,NaN,NaN,None
298416,fs.usda.gov,NaN,NaN,headquarters (not the forest) 701 n. santa ani...,None
343784,motel6.com,NaN,NaN,"2660 forest ln, 75234, dallas, tx, texas",None
180316,premier-stores.co.uk,None,NaN,NaN,None
171138,bdh.vic.gov.au,Boort District Health,NaN,NaN,None
254688,facebook.com,Facebook,NaN,NaN,None
114554,des.az.gov,NaN,Auto Services,NaN,None
126706,bsslawfirm.ca,NaN,NaN,"7071 airport rd, l4t 4j3, mississauga, on, can...",None
